## Cornflakes Indentity Operators

First, we need to make a few trivial operators in the cornflakes DSL. We'll be using the tensorflow DSL, but unfortunately there's no easy way to generate the C-structure for kernels in cornflakes.

In [2]:
import popcorn as pc
Vol_2Vec = pc.DofSpace(2,0,1)
Vol_4Vec = pc.DofSpace(4,0,1)
i_R = pc.Input('iR',Vol_2Vec)
i_K = pc.Input('iK',Vol_4Vec)
o_R = pc.Output('R',(Vol_2Vec,),1)
o_K = pc.Output('K',(Vol_2Vec,),2)
pc.Kernel('idty_R',listing=[
    pc.Asgn(o_R,i_R),
])
pc.Kernel('idty_RK',listing=[
    pc.Asgn(o_R,i_R),
    pc.Asgn(o_K,i_K.reshape(2,2)),

])


Vol_2Vec = pc.DofSpace(2,   0,2)
Face_4Vec = pc.DofSpace(2,  3,4)
Face_16Vec = pc.DofSpace(16,3,4)
i_R = pc.Input('iR',Face_4Vec)
i_K = pc.Input('iK',Face_16Vec)
o_R = pc.Output('R',(Vol_2Vec,),1)
o_K = pc.Output('K',(Vol_2Vec,),2)
pc.Kernel('idty_2_R',listing=[
    pc.Asgn(o_R,i_R),
])
pc.Kernel('idty_2_RK',listing=[
    pc.Asgn(o_R,i_R),
    pc.Asgn(o_K,i_K.reshape(4,4))
])

pc.Husk('identity')

In [ ]:
pc.DofSpace?

In [1]:
import husk_identity

In [2]:
from eoshub import EOSHub

{'water_linear': '/Users/afq/Google Drive/networks/', 'water_iapws_rh_lg': '/Users/afq/Documents/Research/LBNL/eoshub/eoshub/networks/', 'water_iapws_lg': '/Users/afq/Google Drive/networks/', 'water_slgc': '/Users/afq/Google Drive/networks/', 'water_iapws_slgc_logp_comp': '/Users/afq/Google Drive/networks/'}


In [3]:
eos = 'water_iapws_slgc_logp_comp'
network = 'Classifying_pT_0.0_2,6,24,48,sigmoid,0.05'
ls = EOSHub.LatentSim(eos,network)

In [4]:
import cornflakes as cf
import numpy as np
from matplotlib import pylab as plt

Set up the computation graph with cornflakes

In [5]:
Ny = 10
L = 1
X = np.linspace(0,Ny,10)
H_vol = cf.Hypergraph()
for i in range(Ny):
    H_vol.Push_Edge([i])
H_face = cf.Hypergraph()
for i in range(Ny-1):
    H_face.Push_Edge([i,i+1])

Define the cornflakes data and dofmap structures

In [6]:
q = np.zeros(2*Ny)
dm_q = cf.Dofmap(2,0,2)
dm_4 = cf.Dofmap(4,0,4)
data = {'q':(q,dm_q)}

In [8]:
K_all,R_all = ls._sess.run([ls.K_lhs,ls.lhs], feed_dict ={ls.i_q:q.reshape(-1,2)})
R,K = cf.Assemble(husk_identity.kernel_idty_RK,
            H_vol,
            {'iR':(R_all,dm_q),'iK':(K_all,dm_4)},
            {'R':(dm_q,),'K':(dm_q,)},ndof=Ny*2)

In [ ]:
for e in H_vol:
    ii = dm_q.Get_List(e)
    _q = q[ii]
    print(_q)

In [ ]:
rho0,T0 = 200,500
q0 = ls.find_point(rho=rho0,T=T0)
for e in H_vol:
    q[dm_q.Get_List(e)] = q0
    

In [ ]:
def plot(q):
    s = ls.decode(q.reshape(-1,2))
    for i,leg in enumerate(['T','P','rho','rho*h']):
        plt.subplot(1,4,i+1)
        plt.plot(s[:,i],X)
        plt.xlabel(leg)
    plt.tight_layout()
    plt.show()

In [ ]:
plot(q)

In [9]:
# Receives the outputs to recalculate and the input replacements
def replicate_subgraph(outputs, mappings):
    # Types of operation that should not be replicated
    # Taken from tensorflow/python/training/device_setter.py
    NON_REPLICABLE = {'Variable', 'VariableV2', 'AutoReloadVariable',
                      'MutableHashTable', 'MutableHashTableV2',
                      'MutableHashTableOfTensors', 'MutableHashTableOfTensorsV2',
                      'MutableDenseHashTable', 'MutableDenseHashTableV2',
                      'VarHandleOp', 'BoostedTreesEnsembleResourceHandleOp'}
    # Find subgraph ops
    ops = tf.contrib.graph_editor.get_backward_walk_ops(outputs, stop_at_ts=mappings.keys())
    # Exclude non-replicable operations
    ops_replicate = [op for op in ops if op.type not in NON_REPLICABLE]
    # Make subgraph viewitems
    sgv = tf.contrib.graph_editor.make_view(*ops_replicate)
    # Make the copy
    _, info = tf.contrib.graph_editor.copy_with_input_replacements(sgv, mappings)
    # Return new outputs
    return info.transformed(outputs)

In [12]:
import tensorflow as tf

In [14]:
with ls._graph.as_default():
    i_q2 = tf.placeholder(name='i_q2',shape=(None,2),dtype=ls.dtype)
    o_s2 = replicate_subgraph(ls.o_s,{ls.i_q:i_q2})


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [20]:
ls._sess.run([o_s2,ls.o_s],feed_dict={ls.i_q:[q[[0,1]]],i_q2:[[1,0]],})

[array([[1.15844087e+03, 3.19783847e+06, 3.58459091e+00, 3.10831345e+07]]),
 array([[3.55947346e+02, 5.24142647e+04, 6.36222792e+02, 2.25326504e+08]])]

# scratch

In [19]:
X = cf.ParticlePlacers.init_grid(Nx=5,Ny=5,start=[0,0],e1=[L,0],e2=[0,L])
H = cf.Graphers.Build_Pair_Graph(X,cutoff=0.12)

In [21]:
H.view()

[array([[ 0,  2],
        [ 1,  3],
        [ 2,  4],
        [ 3,  5],
        [ 4,  6],
        [ 5,  7],
        [ 6,  8],
        [ 7,  9],
        [ 8, 10],
        [ 9, 11],
        [10, 12],
        [11, 13],
        [14, 12],
        [14, 16],
        [15, 13],
        [15, 17],
        [16, 18],
        [17, 19]], dtype=int32)]

In [29]:
for e in H:
    plt.plot(X[:,e[0]],X[:,e[1]])
plt.show()

IndexError: too many indices for array